In [1]:
import os
import sys
import pandas as pd
#import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..', 'src'))
print(module_path)
# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)
# Now you can import the desired function or class
from graph_helpers import *
from helpers import *
from plots import *
from geopy.distance import geodesic
from find_net_stats import *
from graph_save_run import *
from spatial_helpers import *

/nfs/nfs9/home/nobackup/vakrao/nz_model/src
/nfs/nfs9/home/nobackup/vakrao/nz_model/src


In [2]:
year_log,year_lin = "../src/Y_C/Y_C_log_Y_D.csv","../src/Y_C/Y_C_lin_Y_D.csv"
month_log,month_lin = "../src/M_C/M_C_log_M_D.csv","../src/M_C/M_C_lin_M_D.csv"
seas_log,seas_lin = "../src/S_C/S_C_log_S_D.csv","../src/S_C/S_C_lin_S_D.csv"
real_inc_fp,window_inc_fp  = "../params/new_psa.csv","../params/window_psa.csv"
post_fp = "static_post.csv"
params = "lin"
calib_shift = False
R = .01
ylog_dat = pd.read_csv(year_log)
slog_dat = pd.read_csv(seas_log)
mlog_dat = pd.read_csv(month_log)
drop_subset = ["b_w","b_b","seed","shift","alpha","months","D"]
mlog = mlog_dat.drop_duplicates(subset=drop_subset).reset_index()
slog = slog_dat.drop_duplicates(subset=drop_subset).reset_index()
ylog = ylog_dat.drop_duplicates(subset=drop_subset).reset_index()
ylin_dat = pd.read_csv(year_lin)
slin_dat = pd.read_csv(seas_lin)
mlin_dat = pd.read_csv(month_lin)
month_lin = mlin_dat.drop_duplicates(subset=drop_subset).reset_index()
seas_lin = slin_dat.drop_duplicates(subset=drop_subset).reset_index()
year_lin = ylin_dat.drop_duplicates(subset=drop_subset).reset_index()
#ylog = ylog[ylog["b_b"] > .0001]
#ylog = ylog[ylog["b_w"] > .0001]

In [3]:
#Create graph of all nodes
# now, filter for best fits 
# month best fit: (.04,.1,33297)
# seas best fit: (.03,.08,21709)
# year best fit: (.08,.04,21906)
netfile = "../params/horticulture365_check_NZ.csv"
inc_fp  = "../params/window_psa.csv"
prop_fp = "../params/2024_prop_dat.csv"
post_fp = "temp_post.csv"
eps,pct = 30,0.8
in_bond,out_bond = read_network_data(netfile)
prop_size = read_property_data(prop_fp)
month_real = find_best_config(month_lin,real_inc_fp,"M",eps,pct)
seas_real = find_best_config(seas_lin,real_inc_fp,"S",eps,pct)
year_real = find_best_config(year_lin,real_inc_fp,"Y",eps,pct)
m_bb,m_bw,m_s = month_real["b_b"],month_real["b_w"],str(int(month_real["seed"]))
s_bb,s_bw,s_s = seas_real["b_b"],seas_real["b_w"],str(int(seas_real["seed"]))
y_bb,y_bw,y_s = year_real["b_b"],year_real["b_w"],str(int(year_real["seed"]))
D_y,D_s,D_m =  year_real["D"],seas_real["D"],month_real["D"]
L = 3
T = 425+92
bar_x,bar_y = 6,6
max_infected = 3000
deltaT = 3
min_inc = 1
alpha = 0.1
F = 625
cal_month = 11

In [4]:
print(month_real)
print(seas_real)
print(year_real)

{'b_b': 0.03, 'b_w': 0.09, 'L': 3.0, 'D': 0.4, 'seed': 8878.0, 'RMSE': 75.87, 'inc': [2.0, 1.0, 3.0, 12.0, 22.0, 47.0, 73.0, 111.0, 178.0, 309.0, 198.0, 130.0, 156.0, 153.0], 'total': [2.0, 3.0, 6.0, 18.0, 40.0, 87.0, 160.0, 271.0, 449.0, 758.0, 956.0, 1086.0, 1242.0, 1395.0], 'agg': 'M'}
{'b_b': 0.03, 'b_w': 0.08, 'L': 3.0, 'D': 0.8, 'seed': 21729.0, 'RMSE': 65.83, 'inc': [2.0, 0.0, 2.0, 20.0, 58.0, 57.0, 89.0, 70.0, 103.0, 307.0, 153.0, 106.0, 195.0, 57.0], 'total': [2.0, 2.0, 4.0, 24.0, 82.0, 139.0, 228.0, 298.0, 401.0, 708.0, 861.0, 967.0, 1162.0, 1219.0], 'agg': 'S'}
{'b_b': 0.04, 'b_w': 0.07, 'L': 3.0, 'D': 0.8, 'seed': 10322.0, 'RMSE': 78.26, 'inc': [1.0, 1.0, 0.0, 0.0, 6.0, 15.0, 92.0, 132.0, 185.0, 273.0, 186.0, 144.0, 114.0, 86.0], 'total': [1.0, 2.0, 2.0, 2.0, 8.0, 23.0, 115.0, 247.0, 432.0, 705.0, 891.0, 1035.0, 1149.0, 1235.0], 'agg': 'Y'}


In [5]:
month_s = find_best_config(month_lin,window_inc_fp,"M",eps,pct)
seas_s = find_best_config(seas_lin,window_inc_fp,"S",eps,pct)
year_s = find_best_config(year_lin,window_inc_fp,"Y",eps,pct)

In [6]:
print(month_s)
print(seas_s)
print(year_s)

{'b_b': 0.02, 'b_w': 0.1, 'L': 3.0, 'D': 0.4, 'seed': 21709.0, 'RMSE': 41.23, 'inc': [1.0, 0.0, 1.0, 3.0, 14.0, 55.0, 84.0, 122.0, 151.0, 273.0, 230.0, 124.0, 123.0, 136.0], 'total': [1.0, 1.0, 2.0, 5.0, 19.0, 74.0, 158.0, 280.0, 431.0, 704.0, 934.0, 1058.0, 1181.0, 1317.0], 'agg': 'M'}
{'b_b': 0.03, 'b_w': 0.07, 'L': 3.0, 'D': 0.8, 'seed': 21719.0, 'RMSE': 43.21, 'inc': [1.0, 1.0, 0.0, 11.0, 28.0, 49.0, 94.0, 76.0, 105.0, 244.0, 198.0, 105.0, 156.0, 125.0], 'total': [1.0, 2.0, 2.0, 13.0, 41.0, 90.0, 184.0, 260.0, 365.0, 609.0, 807.0, 912.0, 1068.0, 1193.0], 'agg': 'S'}
{'b_b': 0.02, 'b_w': 0.1, 'L': 3.0, 'D': 0.4, 'seed': 21606.0, 'RMSE': 41.35, 'inc': [2.0, 1.0, 1.0, 3.0, 6.0, 24.0, 45.0, 72.0, 157.0, 241.0, 195.0, 135.0, 99.0, 59.0], 'total': [2.0, 3.0, 4.0, 7.0, 13.0, 37.0, 82.0, 154.0, 311.0, 552.0, 747.0, 882.0, 981.0, 1040.0], 'agg': 'Y'}


In [7]:
t,m_id,m_i = run_save_si_model(in_bond,out_bond,prop_size,m_bb,m_bw,D_m,m_s,T,max_infected,1,min_inc,alpha,F,cal_month)
t,s_id,s_i = run_save_si_model(in_bond,out_bond,prop_size,s_bb,s_bw,D_s,s_s,T,max_infected,3,min_inc,alpha,F,cal_month)
t,y_id,y_i = run_save_si_model(in_bond,out_bond,prop_size,y_bb,y_bw,D_y,y_s,T,max_infected,12,min_inc,alpha,F,cal_month)

In [8]:
for i in in_bond:
    all_vals = in_bond[i]
    for a in all_vals:
        int_a = int(a)
        int_i = int(i)
        if int_i == int_a:
            print("equivalent")

In [9]:
graph_types = ["Avg","Max","Min"]
ret_type = ["Contact)","Monthly)"]

In [10]:
all_data = {}
for g in graph_types:
    for r in ret_type:
        if r == ret_type[0]:
            val = calc_neigh_monthly_spread(s_id,in_bond,prop_fp,g.lower(),int(s_s[0]))
        if r == ret_type[1]:
            val = calc_monthly_spread(s_id,prop_fp,g.lower(),int(s_s[0]))
        all_data[g+"("+r] = val[3:]

In [ ]:
import matplotlib.patches as mpatches
colors = {"Avg":"#fdc086","Max":"#beaed4","Min":"#7fc97f"}
#x_vals = [i for i in range(0,len(neigh_min))]
bar_x,bar_y = 12,6
tick_angle = 30
#x_vals = ["04/11","05/11","06/11","07/11","08/11","09/11","10/11","11/11","12/11","1/12","2/12","3/12"]
months = ["","Mar. 2011","May. 2011","Jul. 2011","Sep. 2011","Nov. 2011","Jan. 2012","Mar. 2012"]

fig,axs = plt.subplots(1,1,figsize=(bar_x,bar_y),constrained_layout=True)
tick_positions = [0, 2, 4, 6, 8, 10, 12]
tick_labels = ["Mar. 2011", "May. 2011", "Jul. 2011", "Sep. 2011", "Nov. 2011", "Jan. 2012", "Mar. 2012"]
leg_fs,title_fs = 14,16
tick_ls = 20
alpha_type = {ret_type[0]:0.4,ret_type[1]:1}
lt = 6
for a in all_data:
    category = a.split("(")
    math_type,val_cat = category[0],category[1]
    y_plot = all_data[a]
    axs.plot(y_plot[1:],color=colors[math_type],alpha=alpha_type[val_cat],lw=lt,label=a)
axs.tick_params(rotation=tick_angle,labelsize=tick_ls)
axs.set_xticks(tick_positions)      # Set the numerical positions of the ticks
axs.set_xticklabels(tick_labels)
axs.set_ylim([.01,100000])
# plot average distance
axs.set_yscale('log')
axs.set_ylabel("Spread Distance (km)",size=24)
#axs.set_xlabel("Time",size=24)
#axs.legend(fontsize=24,bbox_to_anchor=(1, 0.5))
# --- build legend handles manually ---
contact_patches = [
    mpatches.Patch(color=colors["Min"], alpha=0.4, label="Min"),
    mpatches.Patch(color=colors["Avg"], alpha=0.4, label="Avg"),
    mpatches.Patch(color=colors["Max"], alpha=0.4, label="Max")
]

monthly_patches = [
    mpatches.Patch(color=colors["Min"], alpha=1.0, label="Min"),
    mpatches.Patch(color=colors["Avg"], alpha=1.0, label="Avg"),
    mpatches.Patch(color=colors["Max"], alpha=1.0, label="Max")
]

# --- add legends ---
first_legend = axs.legend(handles=contact_patches, 
                          title="Contact Distance", 
                          fontsize=leg_fs, title_fontsize=title_fs,
                          loc="upper left")
axs.add_artist(first_legend)

axs.legend(handles=monthly_patches, 
           title="Detection Distance", 
           fontsize=leg_fs, title_fontsize=title_fs,
           loc="upper center")

plt.savefig("figs/dist_types.png")

Avg(Contact)
[168.0314220528504, 0, 34.7361316574102, 28.612405230957723, 20.034327808413607, 14.903127857585883, 33.29295580791614, 31.395546239887096, 16.546252660292794, 18.156035436341448, 22.836141177266526, 18.67355364811861, 30.175316601079697, 11.746545555626824]
14
Avg(Monthly)
[0, 0, 7.335087979289093, 35.570613044279, 55.62508038544914, 83.94936767180332, 131.20304479536776, 118.89994890105025, 98.83742568946538, 100.02824225865594, 115.43767315065912, 125.19730585632865, 139.40213357020764, 218.437549841437]
14
Max(Contact)
[168.0314220528504, 0, 46.24156083408351, 112.96265024298995, 200.81808358809067, 223.33617514412745, 556.2067794564509, 721.9905056364331, 391.4961704183329, 783.57131632421, 403.8868201814036, 221.75211379448436, 874.684139734356, 54.9866520989895]
14
Max(Monthly)
[0, 0, 7.335087979289093, 230.7902171829145, 222.07495625127459, 263.6307741419777, 556.3908600050204, 790.7973167929417, 620.4122188550888, 1020.808781409049, 723.4164193468057, 657.39199114